[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/castillosebastian/castillosebastian_colabs/blob/main/adult_income_prediction/adult_income_prediction.ipynb)

# Fair IA: An Exploration

## EDA

We are goint to test 'fair-ml' algorithms. We are going to work with *Adult* dataset (Dua & Graff, 2017) used to predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset Train dataset contains 13 features and 30178 observations. Test dataset contains 13 features and 15315 observations. Target column is "target": A binary factor where 1: <=50K and 2: >50K for annual income. The column "sex" is set as protected attribute.

In [10]:
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
names = [
        'age', 'workclass', 'fnlwgt', 'education', 
        'education-num', 'marital-status', 'occupation',
        'relationship', 'race', 'sex', 'capital-gain', 
        'capital-loss', 'hours-per-week', 'native-country',
        'annual-income'
    ]
data = pd.read_csv(url, sep=',', names=names)

In [11]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual-income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
data['annual-income'].value_counts()

annual-income
 <=50K    24720
 >50K      7841
Name: count, dtype: int64

The dataset is imbalanced: 25% make at least $50k per year

This imbalanced is also strongly related to *sex* and *race* as shown here: 

In [17]:
(imbal_sex := data.groupby(['annual-income', 'sex']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')
   .assign(percentage = lambda df:100 * df['count']/df['count'].sum())   
   )

,annual-income,sex,count,percentage
0,<=50K,Male,15128,46.460490
1,<=50K,Female,9592,29.458555
2,>50K,Male,6662,20.460060
3,>50K,Female,1179,3.620896


In [18]:
(imbal_race := data.groupby(['annual-income', 'race']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')
   .assign(percentage = lambda df:100 * df['count']/df['count'].sum())   
   )

,annual-income,race,count,percentage
0,<=50K,White,20699,63.569915
1,>50K,White,7117,21.857437
2,<=50K,Black,2737,8.405761
3,<=50K,Asian-Pac-Islander,763,2.343294
4,>50K,Black,387,1.188538
5,>50K,Asian-Pac-Islander,276,0.847640
6,<=50K,Amer-Indian-Eskimo,275,0.844569
7,<=50K,Other,246,0.755505
8,>50K,Amer-Indian-Eskimo,36,0.110562
9,>50K,Other,25,0.076779


# Simple Nural Network model folowing IBM Research

Source [inFairness](https://github.com/IBM/inFairness/blob/main/examples/adult-income-prediction/adult_income_prediction.ipynb)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from inFairness.fairalgo import SenSeI
from inFairness import distances
from inFairness.auditor import SenSRAuditor, SenSeIAuditor

%load_ext autoreload
%autoreload 2

import data
import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
class AdultDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        return data, label
    
    def __len__(self):
        return len(self.labels)

Note that the categorical variable are transformed into one-hot variables.

In [22]:
train_df, test_df = data.load_data()

X_train_df, Y_train_df = train_df
X_test_df, Y_test_df = test_df

# Let's drop the protected attributes from the training and test data and store them in a
# separate dataframe that we'll use later to train the individually fair metric.
protected_vars = ['race_White', 'sex_Male']

X_protected_df = X_train_df[protected_vars]
X_train_df = X_train_df.drop(columns=protected_vars)
X_test_df = X_test_df.drop(columns=protected_vars)

# Create test data with spouse variable flipped
X_test_df_spouse_flipped = X_test_df.copy()
X_test_df_spouse_flipped.relationship_Wife = 1 - X_test_df_spouse_flipped.relationship_Wife

X_train_df.head()

,age,capital-gain,capital-loss,education-num,hours-per-week,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.409331,-0.14652,-0.218253,-1.613806,-0.496770,False,False,False,False,True,...,False,True,False,False,False,True,False,False,False,False
1,-1.104187,-0.14652,-0.218253,-0.050064,-1.741764,False,False,False,False,True,...,True,False,False,False,False,True,False,False,False,False
2,1.393118,-0.14652,-0.218253,-0.440999,2.574214,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
3,-0.423104,-0.14652,-0.218253,-0.440999,1.163221,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4,-0.877159,-0.14652,-0.218253,1.122743,0.748224,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False


In [23]:
device = torch.device('cpu')

# Convert all pandas dataframes to PyTorch tensors
X_train, y_train = data.convert_df_to_tensor(X_train_df, Y_train_df)
X_test, y_test = data.convert_df_to_tensor(X_test_df, Y_test_df)
X_test_flip, y_test_flip = data.convert_df_to_tensor(X_test_df_spouse_flipped, Y_test_df)
X_protected = torch.tensor(X_protected_df.values).float()

# Create the training and testing dataset
train_ds = AdultDataset(X_train, y_train)
test_ds = AdultDataset(X_test, y_test)
test_ds_flip = AdultDataset(X_test_flip, y_test_flip)

# Create train and test dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1000, shuffle=False)
test_dl_flip = DataLoader(test_ds_flip, batch_size=1000, shuffle=False)

We test a Multilayer neural network as proposed in the IBM implementation example.

In [24]:
class Model(nn.Module):

    def __init__(self, input_size, output_size):

        super().__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fcout = nn.Linear(100, output_size)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fcout(x)
        return x

### Standard training

In [25]:
input_size = X_train.shape[1]
output_size = 2

network_standard = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_standard.parameters(), lr=1e-3)
loss_fn = F.cross_entropy

EPOCHS = 10

In [26]:
network_standard.train()

for epoch in tqdm(range(EPOCHS)):

    for x, y in train_dl:

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = network_standard(x).squeeze()
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

100%|██████████| 10/10 [00:08<00:00,  1.11it/s]


In [27]:
accuracy = metrics.accuracy(network_standard, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_standard, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_standard, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')
print(f'Spouse consistency: {spouse_consistency}')

Accuracy: 0.8522777557373047
Balanced accuracy: 0.7662106720522298
Spouse consistency: 0.9380804953560371


We just drop sensitive column related to *race* and *gender* so why we need another fair-model? It isn't fair yet? The answer is: it is not! Although we drop sensitive columns, the dataset still contains information that give us a strong signal related to the excluded data. For example, 'relationship_Wife'."

The 'spouse consistency' present a 0.7 inconsistency.

[agregar nota]

### Individually fair training with LogReg fair metric in algorithmic way!

In the next section the author build a ML-fair model in the sense that their performance is invariant under certain perturbations in a sensitive subspace, that is: is robust to partial data variations, specifically the parts that are 'sensitive' in fair terms. They explain the motivations of their aproach with an example related to what are called **correspondence studies**: *to investigate whether a resume screening system is fair, an auditor may collect a stack of resumes and change the names on the resumes of Caucasian applicants to names more common among the African-American population. If the system performs worse on the edited resumes, then the auditor may conclude the model treats African-American applicants unfairly. Such investigations are known as correspondence studies.*

The authors propose a method to enforce individual fairness during the training of ML models. They treat fairness as a form of robustness - specifically, robustness to certain sensitive perturbations to the inputs or *sensitive subspace robustness*. This means that the model should still perform well even if the sensitive attributes of the input data are slightly changed: *We cast the fair training problem as training supervised learning systems that are robust to sensitive perturbations* (Yorouchkin, 2020)

To achieve this, the authors use a method called *distributionally robust optimization*. This is a type of optimization that aims to find the best solution under the worst-case distribution of the input data. In this case, the worst-case distribution is one where the sensitive attributes have been perturbed.

The proposed soluctions, named detects aggregate violations of individual fairness. Although the violations are related to indivudal instances an so are individual in nature, the soluctions is only able to detect aggregate violations. These implementations ensures that the ML model is fair not only on the training data but also on new data from the same distribution.

## Learning fair metric from data

Need imputs that can by of three kainds: samples with protected attributes (ex. 'gendres', 'race', etc.), groups of comparable samples and pairs of comparable and incomparable samples. Mahalanobis distance.

Case samples with protectec atributes: 

1- Learn 'sensitive directions' (with Logistic Regression), the result are vector: Vgender, Vrace, etc. 
2- Ignore the 'sensitive directions' in the fair metric. It means tath, when compute de distance do not compute the distance on some dimensions.  Supose that one 'sensitive directions' is gender, we fit a logistric regresion that separete the class (example: men and women), and then use the linear decision boundery (regresion coeficient) to select the vector  ortogonal to that decision boundary and, in a way, in doing that we eliminate separation women-men or turn the clases indecidible.  In fit the logistic regression the algorithm learn also what are good prodictor of gender, and in that way it could find hidden variables that has a strong predictor to gender in order to avoid it in the model. 


# Training

A *variant of adversarial training*: train model accurate on the available data and, importantly, data similar in the fair metric.

Loop:    
    1. Observed data.     
    2. Audit model with DIF: find similar data where algorithm performs differently:    
    3. Update model parameters to minimize prediction error and DIF   
    4. Repeat   

Loss function = Loss functions + DIF
Minimize Loss functions

Idea: Adversarial Robustness



In [30]:
# Same architecture we found
network_fair_LR = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair_LR.parameters(), lr=1e-3)
lossfn = F.cross_entropy

# set the distance metric for instances similiraty detections
distance_x_LR = distances.LogisticRegSensitiveSubspace()
distance_y = distances.SquaredEuclideanDistance()

# train fair metric
distance_x_LR.fit(X_train, data_SensitiveAttrs=X_protected)
distance_y.fit(num_dims=output_size)

distance_x_LR.to(device)
distance_y.to(device)

In [31]:
rho = 5.0
eps = 0.1
auditor_nsteps = 100
auditor_lr = 1e-3

fairalgo_LR = SenSeI(network_fair_LR, distance_x_LR, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)

In [12]:
# these time in traingin, the function to minimaize is the loss functionsn + fair metric (DIF)

fairalgo_LR.train()

for epoch in tqdm(range(EPOCHS)):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo_LR(x, y)
        result.loss.backward()
        optimizer.step()

100%|██████████| 10/10 [09:09<00:00, 54.97s/it]


In [13]:
accuracy = metrics.accuracy(network_fair_LR, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_fair_LR, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_fair_LR, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')
print(f'Spouse consistency: {spouse_consistency}')

Accuracy: 0.8383458852767944
Balanced accuracy: 0.7367161855406339
Spouse consistency: 0.9998894294559929


# Emphasis:

While we remove 'sensitive variables' (sex and race) the model learn what are the other directions related to those sensitive variables and minimace. The model is training to minimace error and remove effect of the sensitive directions. 

### Individually fair training with EXPLORE metric

In [28]:
Y_gender = X_protected[:, -1]
X1, X2, Y_pairs = data.create_data_pairs(X_train, y_train, Y_gender)

distance_x_explore = distances.EXPLOREDistance()
distance_x_explore.fit(X1, X2, Y_pairs, iters=1000, batchsize=10000)
distance_x_explore.to(device)

d:\Users\sebastian.castillo\castillosebastian_colabs\.castillocsebastian_colabs\lib\site-packages\inFairness\distances\explore_distance.py:76: RuntimeWarning: overflow encountered in exp
  sclVec = 2.0 / (np.exp(diag) - 1)


In [32]:
network_fair_explore = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair_explore.parameters(), lr=1e-3)
lossfn = F.cross_entropy

rho = 25.0
eps = 0.1
auditor_nsteps = 10
auditor_lr = 1e-2

fairalgo_explore = SenSeI(network_fair_explore, distance_x_explore, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)

In [33]:
fairalgo_explore.train()

for epoch in tqdm(range(EPOCHS)):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo_explore(x, y)
        result.loss.backward()
        optimizer.step()

100%|██████████| 10/10 [01:17<00:00,  7.71s/it]


In [16]:
accuracy = metrics.accuracy(network_fair_explore, test_dl, device)
balanced_acc = metrics.balanced_accuracy(network_fair_explore, test_dl, device)
spouse_consistency = metrics.spouse_consistency(network_fair_explore, test_dl, test_dl_flip, device)

print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {balanced_acc}')
print(f'Spouse consistency: {spouse_consistency}')

Accuracy: 0.8225342631340027
Balanced accuracy: 0.7034968962244807
Spouse consistency: 1.0


#### Let's now audit the three models and check for their individual fairness compliance

In [17]:
# Auditing using the SenSR Auditor + LR metric

audit_nsteps = 1000
audit_lr = 0.1

auditor_LR = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x_LR, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor_LR.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel_LR = auditor_LR.audit(network_fair_LR, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel_explore = auditor_LR.audit(network_fair_explore, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print("LR metric")
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model - LogReg metric) : {audit_result_fairmodel_LR.lower_bound}. Is model fair: {audit_result_fairmodel_LR.is_model_fair}")
print(f"Loss ratio (fair model - EXPLORE metric) : {audit_result_fairmodel_explore.lower_bound}. Is model fair: {audit_result_fairmodel_explore.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair models are fairer than the standard model")
print("="*100)

/home/sebacastillo/.castillosebastian_colabs/lib/python3.9/site-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in divide
  loss_ratio = np.divide(loss_vals_adversarial, loss_vals_original)


LR metric
Loss ratio (Standard model) : 2.660843321695338. Is model fair: False
Loss ratio (fair model - LogReg metric) : 1.0569381426130153. Is model fair: True
Loss ratio (fair model - EXPLORE metric) : 1.027237853086672. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair models are fairer than the standard model


In [18]:
# Auditing using the SenSR Auditor + EXPLORE metric

audit_nsteps = 1000
audit_lr = 0.1

auditor_explore = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x_explore, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor_explore.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel_LR = auditor_explore.audit(network_fair_LR, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel_explore = auditor_explore.audit(network_fair_explore, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print("EXPLORE metric")
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model - LogReg metric) : {audit_result_fairmodel_LR.lower_bound}. Is model fair: {audit_result_fairmodel_LR.is_model_fair}")
print(f"Loss ratio (fair model - EXPLORE metric) : {audit_result_fairmodel_explore.lower_bound}. Is model fair: {audit_result_fairmodel_explore.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair models are fairer than the standard model")
print("="*100)

EXPLORE metric
Loss ratio (Standard model) : 4.319292031489292. Is model fair: False
Loss ratio (fair model - LogReg metric) : 1.13404923721215. Is model fair: True
Loss ratio (fair model - EXPLORE metric) : 1.0724120239938144. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair models are fairer than the standard model
